## Step 1 - Run this query against your SQL instance to create a database and procedure for use in the stress test

In [27]:
USE master;
GO
DROP DATABASE IF EXISTS ChicagoWhiteSox;
GO
CREATE DATABASE ChicagoWhiteSox;
GO
USE ChicagoWhiteSox;
GO
CREATE OR ALTER PROCEDURE letsgosox
AS
CREATE TABLE #gosox (col1 INT);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:02.707

## Step 2 - Disable in-memory optimization for tempdb by running disableopttempdb.cmd
### The file contains the following code:
### Change the user name, password, and server name  in the cmd file as required
`sqlcmd -Usa -PJu@nS0t022 -i disableopttempdb.sql -S "localhost,1445"
docker stop sqlopttempdb
docker start sqlopttempdb`
### Once changes are made, run the following code from the command line:
`.\disableopttempdb.cmd`

## Step 2 - Execute tempstress.cmd from the command line in the directory that contains the tempdb scripts
### The file contains the following code:
### Change the user name, password, and database as needed
### When ostress completes, note the elapsed time
### You will find it in the command window, as follows: OSTRESS exiting normally, elapsed time: 00:01:08.348
`.\ostress -Usa -PJu@nS0t022 -Q"exec letsgosox" -n50 -r5000 -dChicagoWhiteSox -S"localhost,1445"`
### Once changes are made, run the following code from the command line:
`.\tempstress.cmd`

## Step 3 - While the ostress utility is running run the following query
### This will return the pages that that are being latched
### This will result in PAGELATCH_XX waits

In [24]:
USE tempdb;
GO
SELECT object_name(page_info.object_id) AS object_name, d.wait_type, page_info.* 
FROM sys.dm_exec_requests AS d 
  CROSS APPLY sys.fn_PageResCracker(d.page_resource) AS r
  CROSS APPLY sys.dm_db_page_info(r.db_id, r.file_id, r.page_id,'DETAILED')
    AS page_info;
GO

--DBCC PAGE (2,1,116,3) WITH TABLERESULTS;

Commands completed successfully.

(18 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.204

object_name,wait_type,database_id,file_id,page_id,page_header_version,page_type,page_type_desc,page_type_flag_bits,page_type_flag_bits_desc,page_flag_bits,page_flag_bits_desc,page_lsn,page_level,object_id,index_id,partition_id,alloc_unit_id,is_encrypted,has_checksum,checksum,is_iam_page,is_mixed_extent,has_ghost_records,has_version_records,pfs_page_id,pfs_is_allocated,pfs_alloc_percent,pfs_status,pfs_status_desc,gam_page_id,gam_status,gam_status_desc,sgam_page_id,sgam_status,sgam_status_desc,diff_map_page_id,diff_status,diff_status_desc,ml_map_page_id,ml_status,ml_status_desc,prev_page_file_id,prev_page_page_id,next_page_file_id,next_page_page_id,fixed_length,slot_count,ghost_rec_count,free_bytes,free_bytes_offset,reserved_bytes,reserved_bytes_by_xdes_id,xdes_id
sysschobjs,NULL,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000041:00007230:0151,0,34,1,281474978938880,281474978938880,0,0,696555347,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,301,44,27,0,5288,6906,0,0,0000:00000348
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000041:00007230:0151,0,34,1,281474978938880,281474978938880,0,0,696555347,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,301,44,27,0,5288,6906,0,0,0000:00000348
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000041:00007230:0151,0,34,1,281474978938880,281474978938880,0,0,696555347,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,301,44,27,0,5288,6906,0,0,0000:00000348
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000041:00007230:0151,0,34,1,281474978938880,281474978938880,0,0,696555347,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,301,44,27,0,5288,6906,0,0,0000:00000348
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000041:00007230:0151,0,34,1,281474978938880,281474978938880,0,0,696555347,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,301,44,27,0,5288,6906,0,0,0000:00000348
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000041:00007230:0151,0,34,1,281474978938880,281474978938880,0,0,696555347,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,301,44,27,0,5288,6906,0,0,0000:00000348
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000041:00007230:0151,0,34,1,281474978938880,281474978938880,0,0,696555347,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,301,44,27,0,5288,6906,0,0,0000:00000348
sysschobjs,PAGELATCH_EX,2,1,116,1,1,DATA_PAGE,0x4,,0x0,,00000041:00007230:0151,0,34,1,281474978938880,281474978938880,0,0,696555347,0,1,0,0,1,1,0,0x60,MIXED_EXT | PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,0,0,1,301,44,27,0,5288,6906,0,0,0000:00000348
sysschobjs,PAGELATCH_EX,2,1,301,1,1,DATA_PAGE,0x0,,0x0,,00000041:00007230:0159,0,34,1,281474978938880,281474978938880,0,0,0,0,0,0,0,1,1,0,0x40,PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,1,116,1,300,48,31,0,5264,6314,0,0,0000:00000000
sysschobjs,PAGELATCH_EX,2,1,301,1,1,DATA_PAGE,0x0,,0x0,,00000041:00007230:0159,0,34,1,281474978938880,281474978938880,0,0,0,0,0,0,0,1,1,0,0x40,PFS_IS_ALLOCATED | 0_PCT_FULL,2,1,ALLOCATED,3,0,NOT ALLOCATED,6,1,CHANGED,7,0,NOT MIN_LOGGED,1,116,1,300,48,31,0,5264,6314,0,0,0000:00000000


## Step 4 - Once the ostress utility completes, run the optimizetempdb.cmd file to enable in-memory tempdb system tables

### optimizetempdb.cmd contains the following code

### Change the user name, password, and server name in the cmd file as required

<code>sqlcmd -Usa -PJu@nS0t022 -ioptimizetempdb.sql -Slocalhost,1445 docker stop sqlopttempdb docker start sqlopttempdb</code>

### Once changes are made, execute optimizetempdb.cmd by running the following code at the command line

<code>.\optimizetempdb.cmd</code>

## Step 5 - Execute tempstress.cmd again from the command line in the directory that contains the tempdb scripts
### The file contains the following code:
### Change the user name, password, and database as needed
`ostress -Usa -PJu@nS0t022 -Q"exec letsgosox" -n50 -r5000 -dChicagoWhiteSox -S"localhost,1445"`

## Step 6 - Run the query from Step 3 
### You should not receive any results

In [25]:
USE tempdb;
GO
SELECT object_name(page_info.object_id), d.wait_type, page_info.* 
FROM sys.dm_exec_requests AS d 
  CROSS APPLY sys.fn_PageResCracker(d.page_resource) AS r
  CROSS APPLY sys.dm_db_page_info(r.db_id, r.file_id, r.page_id,'DETAILED')
    AS page_info;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.555

(No column name),wait_type,database_id,file_id,page_id,page_header_version,page_type,page_type_desc,page_type_flag_bits,page_type_flag_bits_desc,page_flag_bits,page_flag_bits_desc,page_lsn,page_level,object_id,index_id,partition_id,alloc_unit_id,is_encrypted,has_checksum,checksum,is_iam_page,is_mixed_extent,has_ghost_records,has_version_records,pfs_page_id,pfs_is_allocated,pfs_alloc_percent,pfs_status,pfs_status_desc,gam_page_id,gam_status,gam_status_desc,sgam_page_id,sgam_status,sgam_status_desc,diff_map_page_id,diff_status,diff_status_desc,ml_map_page_id,ml_status,ml_status_desc,prev_page_file_id,prev_page_page_id,next_page_file_id,next_page_page_id,fixed_length,slot_count,ghost_rec_count,free_bytes,free_bytes_offset,reserved_bytes,reserved_bytes_by_xdes_id,xdes_id


## Step 7 - Find a list of memory-optimized objects
### Run the following query to return the memory optimized objects in tempdb

In [26]:
USE tempdb;
GO
SELECT OBJECT_NAME(object_id) AS ObjectName, * FROM sys.dm_db_xtp_object_stats;
GO


Commands completed successfully.

(12 rows affected)

Total execution time: 00:00:00.214

ObjectName,object_id,xtp_object_id,row_insert_attempts,row_update_attempts,row_delete_attempts,write_conflicts,unique_constraint_violations,object_address
sysrscols,3,-2147483648,1328,0,34,0,0,0x0000001947CC23E0
sysrowsets,5,-2147483647,183,22,7,0,0,0x0000001947CE8040
sysallocunits,7,-2147483646,213,15,7,0,0,0x0000001947F0E040
sysseobjvalues,9,-2147483645,0,0,0,0,0,0x0000001947F56040
sysschobjs,34,-2147483642,2567,20503,4,0,0,0x0000001949906040
sysmultiobjvalues,40,-2147483641,0,0,0,0,0,0x0000001949A9E040
syscolpars,41,-2147483638,1080,1,16,0,0,0x0000001946070040
sysidxstats,54,-2147483637,197,7,7,0,0,0x0000001946142040
sysiscols,55,-2147483636,543,0,9,0,0,0x0000001946194040
sysobjvalues,60,-2147483635,183,0,5,0,0,0x00000019461DC040
